## 5.4 Romberg Integration

Trapezoidal rule: True value of the integral

$$ I = I_i + ch^2_i + O \left( h^4_i \right) $$

Leading-order error

$$ ch^2_i = \frac{1}{3} \left( I_i - I_{i-1} \right) $$

So in other words

$$ I = I_i + \frac{1}{3} \left( I_i - I_{i-1} \right) + O \left( h^4_i \right) $$

as accurately as Simpson's rule. Refining our notation:

$$ R_{i,1} = I_i \quad , \quad R_{i,2} = I_i + \frac{1}{3}(I_i - I_{i-1}) = R_{i,1} + \frac{1}{3} \left( R_{i,1} - R_{i-1,1} \right) $$

then

$$ I = R_{i,2} + c_2 h^4_i + O \left( h^6_i \right) $$

where $ c_2 $ is another constant and $I$ contains only even powers of $ h_i $. Analogously

$$ I = R_{i,2} + \frac{1}{15} \left( R_{i,2} - R_{i-1,2} \right) + O \left( h^6_i \right) $$

In general

$$ I = R_{i, m+1} + O(h^{2m+2}_i) $$

where 

\begin{equation}\label{5.51}\tag{5.51}
    R_{i, m+1} = R_{i, m} + \frac{1}{4^m - 1} \left( R_{i,m} - R_{i-1,m} \right)
\end{equation}

with error

\begin{equation}\label{5.49}\tag{5.49}
    c_m h^{2m}_i = \frac{1}{4^m - 1} \left( R_{i,m} - R_{i-1,m} \right) + O \left( h^{2m+2}_i \right)
\end{equation}

### Algorithm:

1. Calculate $ I_1 \equiv R_{1,1} $ and $ I_2 \equiv R_{2,1} $
2. Calculate $ R_{2,2} $
3. Calculate $ I_3 \equiv R_{3,1} $ and using \eqref{5.51} calculate $ R_{3,2} $ and $ R_{3,3} $
4. At each successive stage, compute $ I_i \equiv R_{i,1} $ and then $ R_{i,2} \dots R_{i,i} $
5. Calculate error with \eqref{5.49} and estimate the integral if some desired target is reached.

<img src='545.png' width="400" height="400">

The equation \eqref{5.49} says the error on $ R_{n,n} $ would be $ \left( R_{n,n} - R_{n-1,n} \right)/(4^n - 1) $, but there is no $ R_{n-1,n} $ so we cannot use the formula in this case. We have to content ourselves with the error estimate for the penultimate entry in each row.

This method is based on a series expansion in powers of $ h $, so it works best in cases where such power series converge rapidly. The integrand should be smooth and "well-behaved".

Cons: Same computer time. Extra programming.

Pros: Accurate to much higher order in $ h $ than the trapezoidal rule (or even Simpson's rule). Less time needed to evaluate integrals.

In [1]:
# Loading packages
import numpy as np
import scipy.integrate as int

In [2]:
# function
def f(x):
    return x**4 - 2*x +1

In [3]:
# trapezoid method with Scipy
a, b = 0.0, 2.0

N = 100
h = (b - a)/N
x = np.arange(a, b+h, h)
y = f(x)

int.trapezoid(y, x)

4.401066656

In [12]:
def trap(N):
    h = (b - a)/N
    x = np.arange(a, b+h, h)
    y = f(x)
    return int.trapezoid(y, x)

# Romberg integration

a, b = 0.0, 2.0
N1 = 1 # Scipy convention
def N(N1, i):
    return N1*2**(i - 1)

def R(i, mp):
    if mp > i:
        return None
    if mp == 1:
        return trap(N(N1, i)) # Ri1
    elif i == 2 and mp == 2:
        return R(2, 1) + (1/3)*(R(2, 1) - R(1, 1)) # R22
    else:
        return R(i, mp - 1) + (1/(4**(mp - 1) - 1))*(R(i, mp - 1) - R(i-1, mp - 1)) # Rim+1

print(R(3, 3))

4.4


In [5]:
# Romberg method with Scipy, convention N1 = 1

function = lambda x: x**4 - 2*x +1
int.romberg(function, a, b, show=True)

Romberg integration of <function vectorize1.<locals>.vfunc at 0x7fd87fa7ec10> from [0.0, 2.0]

 Steps  StepSize   Results
     1  2.000000 14.000000 
     2  1.000000  7.000000  4.666667 
     4  0.500000  5.062500  4.416667  4.400000 
     8  0.250000  4.566406  4.401042  4.400000  4.400000 

The final result is 4.4 after 9 function evaluations.


4.4

In [13]:
# Curiosity

L = lambda a : a*2
L(7)

14

# 5.5 Higher-order integration methods

- Trapezoidal rule: straight-line approx.
- Simpson's rule: quadratic approx.
- Higher-order rules: higher-order polynomial approx. -> Newton-Cotes formulas

In general:

$$ \int_b^a dx f(x) \approx \sum_{k=1}^N w_k f(x_k ) $$

where $x_k$ are the positions of the sample points and $w_k$ are some set of weights.

<img src='5452.png' width="700" height="700">

The $k$th Newton-Cotes rule is exact if the function being integrated is a degree-$k$ polynomial.

In [7]:
# Newton-Cotes method with Scipy, convention N = order

a, b = 0.0, 2.0

N = 10
h = (b - a)/N
x = np.arange(a, b+h, h)
y = x**4 - 2*x +1

an, B = int.newton_cotes(N, 1) #Scipy convention
I = h * np.sum(an * y)

print("""Integral: {0} \nError: {1}""".format(I, I - 4.4))

Integral: 4.4 
Error: 0.0


More ideas:
- We have $N$ sample points, we could just fit one $(N-1)$th-degree polynomial to the whole integration interval, and get an integration method that is exact for $N-1$th-degree polynomials, and for any lower-degree as well.
- It is also possible to derive integration methods that use unevenly spaced points, when is needed to do integrals very fast.
- It might be possible to create an integration rule that is exact for polynomials up to degree $2N-1$.
- Gaussian quadrature.